In [1]:
TRAIN_TFRECORD = 'data/train.tfrecord'
TRAIN_TFRECORD_FILEPATTERN = TRAIN_TFRECORD + '*'
SAVE_DIR = 'models/ddsp-vocal-decoder'

In [2]:
#from ddsp.colab import colab_utils
import ddsp.training
from matplotlib import pyplot as plt
import numpy as np
from misc_util import util

data_provider = ddsp.training.data.TFRecordProvider(TRAIN_TFRECORD_FILEPATTERN)
dataset = data_provider.get_dataset(shuffle=False)

try:
  ex = next(iter(dataset))
except StopIteration:
  raise ValueError(
      'TFRecord contains no examples. Please try re-running the pipeline with '
      'different audio file(s).')

util.specplot(ex['audio'])
util.play(ex['audio'])

f, ax = plt.subplots(3, 1, figsize=(14, 4))
x = np.linspace(0, 4.0, 1000)
ax[0].set_ylabel('loudness_db')
ax[0].plot(x, ex['loudness_db'])
ax[1].set_ylabel('F0_Hz')
ax[1].set_xlabel('seconds')
ax[1].plot(x, ex['f0_hz'])
ax[2].set_ylabel('F0_confidence')
ax[2].set_xlabel('seconds')
ax[2].plot(x, ex['f0_confidence'])

c:\Users\TheoA\Anaconda3\envs\tflow\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import os
os.environ['TF_GPU_ALLOCATOR']='cuda_malloc_async'
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [3]:
!ddsp_run \
  --mode=train \
  --alsologtostderr \
  --save_dir="$SAVE_DIR" \
  --gin_file=models/ae.gin \
  --gin_file=datasets/tfrecord.gin \
  --gin_param="TFRecordProvider.file_pattern='data/train.tfrecord*'" \
  --gin_param="batch_size=16" \
  --gin_param="train_util.train.num_steps=30000" \
  --gin_param="train_util.train.steps_per_save=300" \
  --gin_param="trainers.Trainer.checkpoints_to_keep=10"

^C


In [2]:
from misc_util import util#util.play, util.specplot
import ddsp.training
import gin
from matplotlib import pyplot as plt
import numpy as np

data_provider = ddsp.training.data.TFRecordProvider(TRAIN_TFRECORD_FILEPATTERN)
dataset = data_provider.get_batch(batch_size=1, shuffle=False)

try:
  diter = iter(dataset)
  _ = next(diter)
  _ = next(diter)
  _ = next(diter)
  _ = next(diter)
  _ = next(diter)
  _ = next(diter)
  _ = next(diter)
  _ = next(diter)
  _ = next(diter)
  
  batch = next(diter) # get second example
  print("batch keys", batch.keys(),"batch", batch)
except OutOfRangeError:
  raise ValueError(
      'TFRecord contains no examples. Please try re-running the pipeline with '
      'different audio file(s).')

# # Parse the gin config.
# gin_file = os.path.join(SAVE_DIR, 'operative_config-0.gin')
# gin.parse_config_file(gin_file)

# # Load model
# model = ddsp.training.models.Autoencoder()
# model.restore(SAVE_DIR)

# # Resynthesize audio.
# outputs = model(batch, training=False)
# audio_gen = model.get_audio_from_outputs(outputs)
# audio = batch['audio']

# print('Original Audio')
# util.specplot(audio)
# util.play(audio)

# print('Resynthesis')
# util.specplot(audio_gen)
# util.play(audio_gen)

c:\Users\TheoA\Anaconda3\envs\tflow\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


batch keys dict_keys(['audio', 'audio_16k', 'f0_confidence', 'f0_hz', 'loudness_db']) batch {'audio': <tf.Tensor: shape=(1, 64000), dtype=float32, numpy=
array([[ 0.07019258,  0.05734428,  0.05963317, ..., -0.03863643,
        -0.02020325, -0.02005066]], dtype=float32)>, 'audio_16k': <tf.Tensor: shape=(1, 64000), dtype=float32, numpy=
array([[ 0.07019258,  0.05734428,  0.05963317, ..., -0.03863643,
        -0.02020325, -0.02005066]], dtype=float32)>, 'f0_confidence': <tf.Tensor: shape=(1, 1000), dtype=float32, numpy=
array([[0.9537975 , 0.9549629 , 0.9556276 , 0.9619842 , 0.96243936,
        0.9618254 , 0.9618163 , 0.9626419 , 0.9601169 , 0.9604033 ,
        0.95992184, 0.95958066, 0.95788574, 0.96106195, 0.9615461 ,
        0.95877594, 0.95977455, 0.9610023 , 0.9595969 , 0.9579229 ,
        0.96031785, 0.9566193 , 0.9557281 , 0.95387214, 0.9505844 ,
        0.9469422 , 0.9491522 , 0.9485787 , 0.94678724, 0.95372266,
        0.95457715, 0.95297456, 0.95283145, 0.9515174 , 0.9484024 ,
 